# MNIST DNN & required packages

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets('./dataset/MNIST_data/', one_hot=True)

Extracting ./dataset/MNIST_data/train-images-idx3-ubyte.gz
Extracting ./dataset/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./dataset/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./dataset/MNIST_data/t10k-labels-idx1-ubyte.gz


# Block strategy

### Initial Weights

In [3]:
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init_random_dist)

In [4]:
def init_biases(shape):
    init_random_dist = tf.random_normal(shape, stddev=0.1)
    return tf.Variable(init_random_dist)

### Fully connected layer

In [5]:
def full_connected_layer(input_layer, size, actfn=tf.nn.relu):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_biases([size])
    z = tf.matmul(input_layer, W) + b
    return actfn(z) if actfn else z

### Dropout layer

In [6]:
def dropout_layer(input_layer, keep_prob):
    return tf.nn.dropout(input_layer, keep_prob=keep_prob)

# Architecture

### Placeholder

In [7]:
X = tf.placeholder(tf.float32, shape=[None, 784])

In [8]:
y_true = tf.placeholder(tf.float32, shape=[None, 10])

In [9]:
keep_prob = tf.placeholder(tf.float32)

# Layers

In [10]:
hidden_layer_1 = full_connected_layer(X, 256)
hidden_layer_1 = dropout_layer(hidden_layer_1, keep_prob)

In [11]:
hidden_layer_2 = full_connected_layer(hidden_layer_1, 256)
hidden_layer_2 = dropout_layer(hidden_layer_2, keep_prob)

In [12]:
y_pred = full_connected_layer(hidden_layer_2, 10, None)

# Loss function

In [13]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred))

In [14]:
# regularizers = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2)

# Optimizer

In [15]:
optimizer = tf.train.AdamOptimizer(learning_rate=1e-3)
train = optimizer.minimize(loss)

# Prepare for learning

In [16]:
init = tf.global_variables_initializer()

In [17]:
steps = 5000
batch_sz = 50
dropout = 0.5

with tf.Session() as sess:
    sess.run(init)
    
    for step in range(steps):
        
        x_train_batch, y_train_batch = mnist.train.next_batch(batch_sz)

        sess.run(train, feed_dict={X: x_train_batch, y_true: y_train_batch, keep_prob:dropout})

        if step % 100 == 0:
            matches = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
            accuracy = tf.reduce_mean(tf.cast(matches, tf.float32))
            
            x_test = mnist.test.images
            y_test = mnist.test.labels
            acc, lss, preds = sess.run([accuracy, loss, y_pred], feed_dict={X: x_test, y_true: y_test, keep_prob:dropout})
            print('ON STEM: {:05d}    ACCURACY: {:.5f}    LOSS: {:.5f}'.format(step+1, acc, lss))

ON STEM: 00001    ACCURACY: 0.10350    LOSS: 3.15689
ON STEM: 00101    ACCURACY: 0.73830    LOSS: 0.81328
ON STEM: 00201    ACCURACY: 0.82230    LOSS: 0.56860
ON STEM: 00301    ACCURACY: 0.84730    LOSS: 0.48386
ON STEM: 00401    ACCURACY: 0.87910    LOSS: 0.40665
ON STEM: 00501    ACCURACY: 0.88550    LOSS: 0.37310
ON STEM: 00601    ACCURACY: 0.89450    LOSS: 0.34510
ON STEM: 00701    ACCURACY: 0.90170    LOSS: 0.32586
ON STEM: 00801    ACCURACY: 0.90790    LOSS: 0.30060
ON STEM: 00901    ACCURACY: 0.91130    LOSS: 0.28328
ON STEM: 01001    ACCURACY: 0.91560    LOSS: 0.28194
ON STEM: 01101    ACCURACY: 0.92250    LOSS: 0.26266
ON STEM: 01201    ACCURACY: 0.91850    LOSS: 0.26471
ON STEM: 01301    ACCURACY: 0.92600    LOSS: 0.24784
ON STEM: 01401    ACCURACY: 0.92420    LOSS: 0.25178
ON STEM: 01501    ACCURACY: 0.92880    LOSS: 0.23960
ON STEM: 01601    ACCURACY: 0.92790    LOSS: 0.23868
ON STEM: 01701    ACCURACY: 0.93080    LOSS: 0.22938
ON STEM: 01801    ACCURACY: 0.92900    LOSS: 0

In [18]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report

%matplotlib inline

In [19]:
predictions = np.argmax(preds, axis=1)
y_test = np.argmax(y_test, axis=1)

In [20]:
print(classification_report(y_test, predictions))

             precision    recall  f1-score   support

          0       0.96      0.97      0.97       980
          1       0.98      0.97      0.98      1135
          2       0.95      0.94      0.95      1032
          3       0.94      0.95      0.95      1010
          4       0.94      0.95      0.94       982
          5       0.92      0.93      0.93       892
          6       0.94      0.97      0.96       958
          7       0.97      0.93      0.95      1028
          8       0.93      0.92      0.93       974
          9       0.93      0.93      0.93      1009

avg / total       0.95      0.95      0.95     10000

